# Load Function notebook

In [1]:
#!pip install scikit-surprise

In [2]:
%run Functions.ipynb 

# Model 1: Collab Filtering: KNN with means

## Read the model from saved file

In [3]:
import pickle
knn_filename = 'Model_KNN_Means.sav'
knn_model_full = pickle.load(open(knn_filename, 'rb'))
knn_model=knn_model_full.model

# Model 2: Content Based, Cosine Similarity

## Directly read Femke's notebook

> Important to identify the following:
> cosine_sim and cosine_sim_df

In [4]:
%run 2_ContentBasedFiltering.ipynb 

Shape: (9719, 9719)


# Hybrid Model
Hybrid Recommender leverages the best of both Content based and collaborative filtering techniques.

In [67]:
# Hybrid model
class HybridModel:
    def __init__(self, content_model, cf_model):
        self.content_model = content_model
        self.cf_model = cf_model
        
    def recommend_movies(self, user_title_year, movies_df):
        
        # Use the Process_Avg_Rating function to manipulate the main df and find the 
        # avg rating
        
        movies_df_summary=Process_Avg_Rating(movies_df)
        
        #--------------------------------------
        # Content Based
        #--------------------------------------
        
        # Find the top 100 similar movies based on the content-based model
        similar_movies_cos_sim=self.content_model.genre_recommendation(user_title_year,movies,100)
        
          
        #Merge
        similar_movies_cos_sim_df=pd.merge(similar_movies_cos_sim,movies_df_summary,how='left', left_on=['title','genres'], right_on = ['title','genres'])
        
        #--------------------------------------
        # Col. filter Based
        #--------------------------------------
        
        # Find the top 100 similar movies based on the Coll filter model
        similar_movies_knn=recommend_similar_items(user_input,movies_df,knn_model,100)
        
        #Take the common 10 movies
       
        similar_movies_common=pd.merge(similar_movies_knn,similar_movies_cos_sim_df, how='inner', on=['title','average rating','number of ratings'])
        similar_movies_common=similar_movies_common.sort_values(['average rating'], ascending=[False]).head(10)

        #What if most common movies are <10?
        
        return similar_movies_cos_sim_df, similar_movies_knn,similar_movies_common

## Read the df, and Ask for input

In [57]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
df=pd.read_csv("../../Data/ml-latest-small/PreprocessedData_ml_latest_year_small.csv")
#df["title"] = df["title"].str.lower()
df.head()

,userId,movieId,rating,title,genres,year
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995


In [58]:
#Ask user for the movie name
user_input=AskForUserInput(df)
#Or if you want to hardcode it
#user_input="Toy Story"

Enter your Favorite Movie: Jumanji

We have multiple movies with the same name/Part of it, but with different release years:
 movieId                                 title  year
       2                        Jumanji (1995)  1995
  179401 Jumanji: Welcome to the Jungle (2017)  2017
Which one do you have in your mind? (Enter the movieId)2
Your favourite movie: Jumanji (1995)



## Recommend movies


In [68]:
hybrid_model=HybridModel(model, knn_model)

In [69]:
r_content,r_cf,r_hybrid=hybrid_model.recommend_movies(user_input,df)
print("Recommendations, Content based:")
r_content.head(10)

Recommendations, Content based:


,title,movieId,genres,Similarity Score,average rating,number of ratings
0,"Water Horse: Legend of the Deep, The (2007)",56915,Adventure|Children|Fantasy,1.0,4.00,1
1,Bridge to Terabithia (2007),50601,Adventure|Children|Fantasy,1.0,2.78,9
2,Gulliver's Travels (1996),173873,Adventure|Children|Fantasy,1.0,2.00,1
3,Percy Jackson: Sea of Monsters (2013),104074,Adventure|Children|Fantasy,1.0,1.75,2
4,"Chronicles of Narnia: Prince Caspian, The (2008)",59501,Adventure|Children|Fantasy,1.0,3.54,12
5,Pan (2015),130450,Adventure|Children|Fantasy,1.0,2.50,3
6,"Indian in the Cupboard, The (1995)",60,Adventure|Children|Fantasy,1.0,3.24,34
7,Alice Through the Looking Glass (2016),158813,Adventure|Children|Fantasy,1.0,2.00,4
8,Harry Potter and the Sorcerer's Stone (a.k.a. ...,4896,Adventure|Children|Fantasy,1.0,3.76,107
9,"NeverEnding Story III, The (1994)",126,Adventure|Children|Fantasy,1.0,2.00,7


In [70]:
print("Recommendations based on CF:")
r_cf.head(10)

Recommendations based on CF:


,,,,average rating,number of ratings
movieId,title,year,genres,,
1,Toy Story (1995),1995,Adventure|Animation|Children|Comedy|Fantasy,3.92,215
21,Get Shorty (1995),1995,Comedy|Crime|Thriller,3.49,89
24,Powder (1995),1995,Drama|Sci-Fi,3.12,28
36,Dead Man Walking (1995),1995,Crime|Drama,3.84,67
48,Pocahontas (1995),1995,Animation|Children|Drama|Musical|Romance,3.15,68
62,Mr. Holland's Opus (1995),1995,Drama,3.71,80
95,Broken Arrow (1996),1996,Action|Adventure|Thriller,3.02,84
150,Apollo 13 (1995),1995,Adventure|Drama|IMAX,3.85,201
158,Casper (1995),1995,Adventure|Children,2.81,62


In [71]:
print("Recommendations, naive hybrid:")

r_hybrid.head(10)

Recommendations, naive hybrid:


,title,average rating,number of ratings,movieId,genres,Similarity Score
0,Toy Story (1995),3.92,215,1,Adventure|Animation|Children|Comedy|Fantasy,0.813523
2,Atlantis: The Lost Empire (2001),3.34,19,4366,Adventure|Animation|Children|Fantasy,0.844296
1,Casper (1995),2.81,62,158,Adventure|Children,0.804634
